In [102]:
import sympy as sym
import numpy as np
np.set_printoptions(linewidth=200, formatter={'float': '{:.3e}'.format})
import matplotlib.pyplot as plt

In [116]:
def approximate_Neumann_inversion(A,n):
    I = sym.eye(A.shape[0])
    A_inv_approx = sym.zeros(A.shape[0])
    for i in range(n+1):
        A_inv_approx += (I-A)**i
    return A_inv_approx

In [159]:
def approximate_taylor_inversion(D,R,ε,n):
    
    if n == 0:
        return D ** -1
    elif n == 1:
        return D ** -1 - ε * R * D**-2
    elif n == 2:
        return D ** -1 - ε * R * D**-2 +  ε**2 * R*2 * D**-3
        

In [210]:
def block_inversion(M):
    A = M[:3,:3]
    D = M[3:,3:]
    C = M[3:,:3]
    A_inv = A**-1
    print('Inverse A ready')
    S = D - C * A_inv * C.T
    S_inv = S**-1
    print('Inverse S ready')
    M_inv = sym.zeros(6)
    M_inv[:3,:3] = A_inv + A_inv * C.T * S_inv * C * A_inv
    M_inv[3:,3:] = S_inv
    M_inv[3:,:3] = -S_inv * C * A_inv
    M_inv[:3,3:] = -A_inv * C.T * S_inv
    return M_inv

In [103]:
Φp_a, Φm_a, Φp_b, Φm_b, Φp_c, Φm_c = sym.symbols('\dot{\Phi}_{+A}, \dot{\Phi}_{-A}, '
                                                 '\dot{\Phi}_{+B}, \dot{\Phi}_{-B}, '
                                                 '\dot{\Phi}_{+C}, \dot{\Phi}_{-C}', real=True, positive=True, nonzero=True)
Cp_a, Cm_a, Cp_b, Cm_b, Cp_c, Cm_c, Cc = sym.symbols(   'C_{+A}, C_{-A},'
                                                        'C_{+B}, C_{-B},'
                                                        'C_{+C}, C_{-C}, C_C', real=True, positive=True, nonzero=True)

In [104]:
Cp_a_val, Cm_a_val, Cp_b_val, Cm_b_val, Cp_c_val, Cm_c_val, Cc_val = [14,8,7,9,11,13,1]

In [105]:
Φ_vec = sym.Matrix([Φp_a, Φm_a, Φp_b, Φm_b, Φp_c, Φm_c])
C_vec = sym.Matrix([Cp_a, Cm_a, Cp_b, Cm_b, Cp_c, Cm_c])
C_vec_simple = sym.Matrix([Cp_a, Cm_a, Cp_a, Cm_a, Cp_a, Cm_a])

In [106]:
Φ1_a = 1/2*(Φp_a+Φm_a)
Φ2_a = 1/2*(Φp_a-Φm_a)
Φ1_b = 1/2*(Φp_b+Φm_b)
Φ2_b = 1/2*(Φp_b-Φm_b)
Φ1_c = 1/2*(Φp_c+Φm_c)
Φ2_c = 1/2*(Φp_c-Φm_c)

In [107]:
# Change of variables from (0,1) to (p,m)
P1 = 0.5 *sym.Matrix([[1,  1  ,0  , 0  ,0  , 0  ],
                      [1, -1  ,0  , 0  ,0  , 0  ],
                      [0,  0  ,1  , 1  ,0  , 0  ],
                      [0,  0  ,1  ,-1  ,0  , 0  ],
                      [0,  0  ,0  ,0   ,1  , 1  ],
                      [0,  0  ,0  ,0   ,1  ,-1  ]])

## Asymmetric capacitive coupling between two unit cells (only inner nodes) 

In [108]:
0.5 * Cc * ( (Φ2_a-Φ1_b)**2 + (Φ1_c-Φ2_b)**2 )

0.5*C_C*(0.25*(\dot{\Phi}_{+A} - \dot{\Phi}_{+B} - \dot{\Phi}_{-A} - \dot{\Phi}_{-B})**2 + 0.25*(-\dot{\Phi}_{+B} + \dot{\Phi}_{+C} + \dot{\Phi}_{-B} + \dot{\Phi}_{-C})**2)

In [109]:
Cc_mat_01 = sym.Matrix([[0,  0  ,0  , 0  ,0  , 0  ],
                       [0, Cc  ,-Cc, 0  ,0  , 0  ],
                       [0,-Cc  ,Cc , 0  ,0  , 0  ],
                       [0,  0  ,0  , Cc ,-Cc, 0  ],
                       [0,  0  ,0  ,-Cc ,Cc , 0  ],
                       [0,  0  ,0  ,0   ,0  , 0  ]])

In [110]:
Cc_mat_pm = P1.T * Cc_mat_01 * P1
Cc_mat_pm

Matrix([
[ 0.25*C_C, -0.25*C_C, -0.25*C_C, -0.25*C_C,         0,         0],
[-0.25*C_C,  0.25*C_C,  0.25*C_C,  0.25*C_C,         0,         0],
[-0.25*C_C,  0.25*C_C,   0.5*C_C,         0, -0.25*C_C, -0.25*C_C],
[-0.25*C_C,  0.25*C_C,         0,   0.5*C_C,  0.25*C_C,  0.25*C_C],
[        0,         0, -0.25*C_C,  0.25*C_C,  0.25*C_C,  0.25*C_C],
[        0,         0, -0.25*C_C,  0.25*C_C,  0.25*C_C,  0.25*C_C]])

In [111]:
sym.simplify( (Φ_vec.T * Cc_mat_pm * Φ_vec)[0] )

C_C*(-0.25*\dot{\Phi}_{+A}*(-\dot{\Phi}_{+A} + \dot{\Phi}_{+B} + \dot{\Phi}_{-A} + \dot{\Phi}_{-B}) - \dot{\Phi}_{+B}*(0.25*\dot{\Phi}_{+A} - 0.5*\dot{\Phi}_{+B} + 0.25*\dot{\Phi}_{+C} - 0.25*\dot{\Phi}_{-A} + 0.25*\dot{\Phi}_{-C}) + 0.25*\dot{\Phi}_{+C}*(-\dot{\Phi}_{+B} + \dot{\Phi}_{+C} + \dot{\Phi}_{-B} + \dot{\Phi}_{-C}) + 0.25*\dot{\Phi}_{-A}*(-\dot{\Phi}_{+A} + \dot{\Phi}_{+B} + \dot{\Phi}_{-A} + \dot{\Phi}_{-B}) + \dot{\Phi}_{-B}*(-0.25*\dot{\Phi}_{+A} + 0.25*\dot{\Phi}_{+C} + 0.25*\dot{\Phi}_{-A} + 0.5*\dot{\Phi}_{-B} + 0.25*\dot{\Phi}_{-C}) + 0.25*\dot{\Phi}_{-C}*(-\dot{\Phi}_{+B} + \dot{\Phi}_{+C} + \dot{\Phi}_{-B} + \dot{\Phi}_{-C}))

In [112]:
sym.simplify( (0.5 * Φ_vec.T * Cc_mat_pm * Φ_vec)[0] - 0.5 * Cc * ( (Φ2_a-Φ1_b)**2 + (Φ1_c-Φ2_b)**2 ))

0

In [226]:
C0_mat_pm = sym.diag(*C_vec)
C0_mat_pm_simple = sym.diag(*C_vec_simple)

In [227]:
C_mat_pm = C0_mat_pm + Cc_mat_pm
C_mat_pm_simple = C0_mat_pm_simple + Cc_mat_pm

In [211]:
C_mat_pm_inv_approx = approximate_Neumann_inversion(C_mat_pm/Cp_a,1)
C_mat_pm_inv_approx_num = np.array( (Cp_a*C_mat_pm_inv_approx).subs({Cp_a:Cp_a_val, Cm_a:Cm_a_val, Cp_b:Cp_b_val, Cm_b:Cm_b_val, Cp_c:Cp_c_val, Cm_c:Cm_c_val, Cc:Cc_val }).evalf() ).astype('float')

In [212]:
D = sym.diag(*C_mat_pm.diagonal())
R = C_mat_pm - D
R /= Cc
D /= Cp_a
ε = Cc / Cp_a

In [156]:
C_mat_pm_inv_approx = approximate_taylor_inversion(D,R,ε,1)
C_mat_pm_inv_approx_num = np.array( (Cp_a*C_mat_pm_inv_approx).subs({Cp_a:Cp_a_val, Cm_a:Cm_a_val, Cp_b:Cp_b_val, Cm_b:Cm_b_val, Cp_c:Cp_c_val, Cm_c:Cm_c_val, Cc:Cc_val }).evalf() ).astype('float')

In [213]:
C_mat_pm_inv_approx = block_inversion(C_mat_pm)
C_mat_pm_inv_approx_num = np.array( (Cp_a*C_mat_pm_inv_approx).subs({Cp_a:Cp_a_val, Cm_a:Cm_a_val, Cp_b:Cp_b_val, Cm_b:Cm_b_val, Cp_c:Cp_c_val, Cm_c:Cm_c_val, Cc:Cc_val }).evalf() ).astype('float')

Inverse A ready


KeyboardInterrupt: 

In [161]:
C_mat_pm_num = np.array( C_mat_pm.subs({Cp_a:Cp_a_val, Cm_a:Cm_a_val, Cp_b:Cp_b_val, Cm_b:Cm_b_val, Cp_c:Cp_c_val, Cm_c:Cm_c_val, Cc:Cc_val }).evalf() ).astype('float')
C_mat_pm_inv_num = np.linalg.inv(C_mat_pm_num)

In [162]:
C_mat_pm_inv_num

array([[7.028e-02, 2.006e-03, 2.277e-03, 1.796e-03, 1.048e-05, 8.864e-06],
       [2.006e-03, 1.215e-01, -3.984e-03, -3.143e-03, -1.833e-05, -1.551e-05],
       [2.277e-03, -3.984e-03, 1.337e-01, 2.307e-05, 2.916e-03, 2.468e-03],
       [1.796e-03, -3.143e-03, 2.307e-05, 1.055e-01, -2.301e-03, -1.947e-03],
       [1.048e-05, -1.833e-05, 2.916e-03, -2.301e-03, 8.904e-02, -1.582e-03],
       [8.864e-06, -1.551e-05, 2.468e-03, -1.947e-03, -1.582e-03, 7.558e-02]])

In [164]:
C_mat_pm_inv_approx_num/100

array([[1.375e-01, 2.500e-03, 2.500e-03, 2.500e-03, 0.000e+00, 0.000e+00],
       [2.500e-03, 1.975e-01, -2.500e-03, -2.500e-03, 0.000e+00, 0.000e+00],
       [2.500e-03, -2.500e-03, 2.050e-01, 0.000e+00, 2.500e-03, 2.500e-03],
       [2.500e-03, -2.500e-03, 0.000e+00, 1.850e-01, -2.500e-03, -2.500e-03],
       [0.000e+00, 0.000e+00, 2.500e-03, -2.500e-03, 1.675e-01, -2.500e-03],
       [0.000e+00, 0.000e+00, 2.500e-03, -2.500e-03, -2.500e-03, 1.475e-01]])